In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import logging

logging.getLogger('jieba').setLevel(logging.WARN)
logging.getLogger('fgclassifier').setLevel(logging.INFO)

In [2]:
import os

os.chdir('..')

In [5]:
import config
from collections import defaultdict
from sklearn.model_selection import train_test_split

from fgclassifier.utils import read_data, get_dataset

# The English data set is the first 10000 reviews,
# we make it consistent here
X_train, y_train = read_data(get_dataset('train')[:10000])
X_train.shape

(10000,)

In [ ]:
from IPython.display import clear_output

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib

from fgclassifier.baseline import Baseline, MultiOutputClassifier
from fgclassifier.features import Count, Tfidf, SVD, SparseToDense
from fgclassifier.classifiers import ComplementNB

pipeline = Pipeline([
    ('vect', Count(min_df=0.02, max_df=0.99, ngram_range=(1, 5))),
    ('Logistic', ComplementNB()),
], memory='data/pipeline_cache')

clf = MultiOutputClassifier(LinearDiscriminantAnalysis())
parameters = {
    'vect__min_df': [0.05, 0.1, 0.2, 0.3, 0.5],
}

with joblib.parallel_backend('threading', n_jobs=1):
    grid_search = GridSearchCV(
        clf, parameters, cv=4,
        verbose=True, return_train_score=True
    )
    grid_search.fit(Xt, y_train)
    
clear_output()